##### **Step 3: In groups, create a Jupyter notebook that shows the mortgage problem solved in Python. Make sure to:**
* Illustrate how your data structure is used to solve the problem from GWP 1.
* Ensure that your results match the results from the spreadsheet.


In [1]:
import pandas as pd
import numpy as np
import datetime

pd.set_option('display.precision',2)
pd.set_option('styler.format.thousands','.')

#### Cashflow is calcualted by formular:

$$\text{Cashflow} =\frac{\mathrm{Principal} \times \mathrm{Ratemonthly}}{1-\frac{1}{(1+\mathrm{Ratemonthly})^{\mathrm{Year}\times 12}}}$$


# New

In [2]:
def mortgage(year, rate=0, start_date=None, principal=1000000):

    period = 12*year
    remainingprincipal = 0

    if start_date is None:
        pass
    else:
        start_date = pd.to_datetime(start_date)
        # initial array of period in monthly from start date
        rangedatemonth = pd.date_range(
            start=start_date-pd.DateOffset(months=1), periods=period+1, freq='MS')

        ratedf = pd.read_csv("https://docs.google.com/spreadsheets/d/e/"
                             "2PACX-1vS5svAeBupJJ5CsK94DPLLTTVB4ZuckMMi-suaBzzajdfbe2"
                             "amCffYEz9KysCItneZ7Jk65eesKFWTN/pub?output=csv"
                             )
        ratedf = ratedf.iloc[:, 1:3]
        ratedf.DATE = pd.to_datetime(ratedf.DATE)
        ratedf.sort_values("DATE", inplace=True)
        ratedf['Year'] = pd.DatetimeIndex(ratedf.DATE).year

        raterange = []
        for d in rangedatemonth:
            if d.year < (start_date.year+7):
                raterange.append(
                    ratedf[~(ratedf['Year'] >= start_date.year)][-1:].MORTGAGE30US.to_list()[0]/100)
            else:
                # year duration - 7 years
                raterange.append(
                    ratedf[~(ratedf['Year'] >= d.year)][-1:].MORTGAGE30US.to_list()[0]/100)

    frame = []
    
    for p in range(period+1):
        openningprincipal = remainingprincipal
        if start_date != None:
            rate = raterange[p]

        if p == 0:
            cashflow = 0
            interestpaid = 0
            principalpaid = cashflow-interestpaid
            remainingprincipal = principal
            frame.append([p, principal, rate, cashflow,
                         interestpaid, principalpaid, remainingprincipal])
        else:
            if (p >= 12*7) and (p % 12 == 1):
                principal = remainingprincipal
                period = year*12 - p + 1
            if period == 0:
                cashflow = openningprincipal
            else:
                cashflow = (principal * rate/12) / \
                    (1 - 1/((1+rate/12)**(period)))
            interestpaid = openningprincipal*rate/12
            principalpaid = cashflow-interestpaid
            remainingprincipal = openningprincipal-principalpaid
            frame.append([p, openningprincipal, rate, cashflow,
                         interestpaid, principalpaid, remainingprincipal])

    df = pd.DataFrame(frame, columns=["Month", "Opening Principal Balance", "Rates", "Cashflow",
                                      "Interest Paid", "Principal Paid", "Closing Princial Balance"])
    df.set_index("Month", inplace=True)

    return df


#### **Mortgage 1:**
* Time: 30 years
* Rate: 4%
* Principal Amount: $1,000,000

In [3]:
Mortgage1 = mortgage(30,0.04)
print("Total Interest Paid: ${:#,.2f}".format(Mortgage1['Interest Paid'].sum()))
Mortgage1

Total Interest Paid: $718,695.06


,Opening Principal Balance,Rates,Cashflow,Interest Paid,Principal Paid,Closing Princial Balance
Month,,,,,,
0,1000000.00,0.04,0.00,0.00,0.00,1.00e+06
1,1000000.00,0.04,4774.15,3333.33,1440.82,9.99e+05
2,998559.18,0.04,4774.15,3328.53,1445.62,9.97e+05
3,997113.56,0.04,4774.15,3323.71,1450.44,9.96e+05
4,995663.12,0.04,4774.15,3318.88,1455.28,9.94e+05
...,...,...,...,...,...,...
356,23633.90,0.04,4774.15,78.78,4695.37,1.89e+04
357,18938.53,0.04,4774.15,63.13,4711.02,1.42e+04
358,14227.50,0.04,4774.15,47.43,4726.73,9.50e+03


#### **Mortgage 2:**
* Time: 20 years
* Rate: 2.5%
* Principal Amount: $1,000,000

In [4]:
Mortgage2 = mortgage(20,0.025)
print("Total Interest Paid: ${:#,.2f}".format(Mortgage2['Interest Paid'].sum()))
Mortgage2

Total Interest Paid: $271,766.94


,Opening Principal Balance,Rates,Cashflow,Interest Paid,Principal Paid,Closing Princial Balance
Month,,,,,,
0,1000000.00,0.03,0.00,0.00,0.00,1.00e+06
1,1000000.00,0.03,5299.03,2083.33,3215.70,9.97e+05
2,996784.30,0.03,5299.03,2076.63,3222.39,9.94e+05
3,993561.91,0.03,5299.03,2069.92,3229.11,9.90e+05
4,990332.80,0.03,5299.03,2063.19,3235.84,9.87e+05
...,...,...,...,...,...,...
236,26330.35,0.03,5299.03,54.85,5244.17,2.11e+04
237,21086.18,0.03,5299.03,43.93,5255.10,1.58e+04
238,15831.08,0.03,5299.03,32.98,5266.05,1.06e+04


#### **Mortgage 3:**
* Time: 30 years
* Rate: 7-1 Adjustable
* Principal Amount: $1,000,000

In [5]:
Mortgage3 = mortgage(year=30,start_date='1990-01-01')
print("Total Interest Paid: ${:#,.2f}".format(Mortgage3['Interest Paid'].sum()))
Mortgage3

Total Interest Paid: $1,488,119.85


,Opening Principal Balance,Rates,Cashflow,Interest Paid,Principal Paid,Closing Princial Balance
Month,,,,,,
0,1000000.00,0.10,0.00,0.00,0.00,1.00e+06
1,1000000.00,0.10,8613.58,8150.00,463.58,1.00e+06
2,999536.42,0.10,8613.58,8146.22,467.36,9.99e+05
3,999069.05,0.10,8613.58,8142.41,471.17,9.99e+05
4,998597.88,0.10,8613.58,8138.57,475.01,9.98e+05
...,...,...,...,...,...,...
356,29374.18,0.05,5941.83,111.38,5830.45,2.35e+04
357,23543.73,0.05,5941.83,89.27,5852.56,1.77e+04
358,17691.17,0.05,5941.83,67.08,5874.75,1.18e+04
